Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1373]:
from random import choices
import numpy as np
import lab9_lib
import copy

In [1386]:
class individual:
    
    def __init__(self, ind, fitness) -> None:
        self.ind = ind
        self.fitness = fitness
    
    def get_fitness(self):
        return self.fitness
    
    def set_fitness(self, fitness):
        self.fitness = fitness

    def get_ind(self):
        return self.ind

    def tweak(self):
        #print(f"fitnes: {self.fitness}")
        num_tweaks = int((10-np.floor((self.fitness * 10))))
        #print(f"num tweaks {num_tweaks}")
        for n in range(num_tweaks):
            #print
            index = np.random.randint(0,len(self.ind))
            #print(f"index = {index}")
            self.ind[index] = 1 if self.ind[index] == 0 else 0



    def __str__(self):
         return f"{''.join(str(g) for g in self.ind)}: {self.get_fitness():.2%}"

In [1387]:
class population:

    def __init__(self, individuals: list[individual], fitness) -> None:
        self.individuals = individuals
        self.fitness = fitness
    
    def make_new_generation(self):
        new_population = []
        for individual in self.individuals:
            new_individual = copy.deepcopy(individual)
            new_individual_before_tweak = copy.deepcopy(new_individual)
            #print(f"copy: {new_individual}")
            new_individual.tweak()

            tweaked_fitness = self.fitness(new_individual.ind)
            new_individual.set_fitness(tweaked_fitness)
            #print(f"new_individual: {new_individual}")   
            #print(f"new fitness: {tweaked_fitness}")
            #print(f"new and old the same: {new_individual.ind == new_individual_before_tweak.ind}")
            new_population.append(new_individual)
        return new_population
    
    def pluss_strategy(self):
        #print(f"old generation: {self}")
        offspring = self.make_new_generation()
        #print("new generation")
        #print("\n")
        new_generation = self.individuals + offspring
        new_generation = sorted(new_generation, key=lambda i: i.get_fitness(), reverse=True)[:10]
        #print(new_generation)

        return new_generation
    
    def update_population(self):
        self.individuals = self.pluss_strategy()
    
    def __str__(self):
        return ''.join(i.__str__() + "\n" for i in self.individuals)

    def find_best_individual(self):
        return max(self.individuals, key=lambda i: i.fitness)

In [1388]:
fitness = lab9_lib.make_problem(10)
individuals = []
for n in range(10):
    ind = choices([0, 1], k=50)
    i = individual(ind, fitness(ind))
    individuals.append(i)
    print(f"{''.join(str(g) for g in ind)}: {fitness(i.ind):.2%}")


10001110000001001111010000001000110100000101100000: 5.56%
11000011001111011010011111000100101100100001111101: 7.33%
10110001011101011100010111111101010010011010011101: 31.33%
00000111001101011110101010011101001111111110011010: 31.34%
01010110000110010100011001011101100111001000010110: 9.11%
00110110100010110001001010111110100000110000100101: 15.34%
01111001110001001011101111111111100110110110011100: 31.33%
00010001011110010100001100101101000101101010100111: 7.33%
10011011010111010101111110101010100011101111111100: 39.34%
01000110111100010111110000001010010000001111011011: 23.34%


In [1389]:
def make_new_generation(old_population: list[individual]):
    new_population = []
    for individual in old_population:
        new_individual = copy.deepcopy(individual)
        new_individual.tweak()
        new_individual.set_fitness(fitness(new_individual.ind))
        new_population.append(new_individual)

    
    return new_population

In [1390]:
# p = population(individuals, fitness)
# print(p)
# p.update_population()
# print(p)
# p.update_population()
# print(p)

In [1417]:
def simulate(n, k):
    fitness = lab9_lib.make_problem(100)
    individuals = []
    for n in range(10):
        ind = choices([0, 1], k=k)
        i = individual(ind, fitness(ind))
        individuals.append(i)

    pop = population(individuals, fitness)
    best_individual = pop.find_best_individual()
    print(best_individual)
    generations = 0
    while best_individual.get_fitness() < 1.0 and generations < 10000:
        #print(pop)
        pop.update_population()
        generations += 1 
        if pop.find_best_individual().fitness > best_individual.fitness:
            best_individual = pop.find_best_individual()
    print(f"generations: {generations}")
    print(f"fitness calls: {fitness.calls}")
    return best_individual
    

In [1418]:
best_individual = simulate(10, 100)

0000100010111011100100111000111010111111010110011011011110001101111110011111101101101011111001001000: 59.00%
1110010001010000011010011110000001100000011001101110110111101001101101110101111110010011000110001110: 51.00%
0100101010111011100100111000111010111111010110011011011110011101111110011111101100101011111000001000: 60.00%
0100110100100100011011101000111111100110010011010110111000010100011100101100111100011010011010110001: 51.00%
1000011100100100010010110000111000001001111111111101111000001010000110101100110100111010111101011111: 53.00%
0000101100010101001101011010011000101111111011011001000001011111001110000010001010101010001001101010: 47.00%
1010010111011100000010111000000100111101110001000011001101000000001111110000110101000100011101111001: 46.00%
0001000110111111000010111001100000010000011000000111000111110111111110111010010000000101101111010101: 49.00%
1111001101011101001111011111111001111000100011011011101111000010110000001000011001111011011011011110: 59.00%
1100101111100100111

In [1414]:
best_individual.fitness

1.0